# Truss Analyer Testing Zone

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import re
import math
import pandas as pd
from PyNite import FEModel3D
from PyNite.Visualization import render_model
from rich import print
import matplotlib.pyplot as plt
# import truss_input as ti
import truss_model as tm

In [5]:
# top_nodes = []
# bot_nodes = []
# d_w = L/(n/2)
# for i, loc in enumerate(list(np.linspace(0, L, int(n/2+1)))):
#     t_node = [loc, d]
#     top_nodes.append(t_node)
#     if i == 0:
#         continue
#     else: 
#         b_node = [loc-d_w/2, 0] 
#         bot_nodes.append(b_node)

In [7]:
# Import and clean Canam's tables

def remove_comma(cell):
    number = float(cell.replace(',', ''))
    return number

canam_df = pd.read_csv('OWSJ_tables_canam.csv',header=1)
canam_df.rename(columns={canam_df.columns[0]: 'span', canam_df.columns[1]: 'depth'}, inplace=True)
canam_df['span'].ffill(inplace=True)
canam_df['span']= canam_df['span']*1000
canam_df['depth'] = canam_df['depth'].apply(remove_comma)
canam_df['depth'] = canam_df['depth'].astype(float)
canam_df.head(10)

# save as json for linux reading
# canam_df.to_json('canam_df.json', orient='records')

In [8]:
# Import Vulcraft's data
vulcraft_df = pd.read_csv('OWSJ_tables_vulcraft.csv', header=1)
vulcraft_df.rename(columns={"Factored Load": 'value'}, inplace=True)
vulcraft_df.columns = [col.lower() for col in vulcraft_df]
#drop first row and all bridging rows
vulcraft_df.drop(index=0, inplace=True)
vulcraft_df = vulcraft_df[~vulcraft_df['value'].str.contains('Bridg.')]
#fill depths
vulcraft_df['depth'].bfill(inplace=True)
#convert all span values to mm
vulcraft_df.span = vulcraft_df.span.astype(float)
vulcraft_df.span = vulcraft_df.span*1000
vulcraft_df.head(30)

# save as json for linux reading
# vulcraft_df.to_json('vulcraft_df.json', orient='records')

In [9]:
# Import Omega's tables

def find_span(cell):
    if "SPAN" in cell:
        match_span = re.search(r'\b(\d+)\b', cell)
        return match_span.group(1)
    else:
        return np.nan

omega_df = pd.read_csv('OWSJ_tables_omega.csv', header=None)
col_names = ["depth", "D/D", "E/D", "E/E", "F/E", "F/F", "G/F", "G/G", "H/G", "H/H", "K/H", "K/K", "L/K", "L/L"]
omega_df.columns = col_names
omega_df['span'] = omega_df['depth'].apply(find_span)
omega_df['span'].ffill(inplace=True)
columns = ['span'] + [col for col in omega_df if col != 'span']
omega_df = omega_df[columns]
# Drop rows with useless info; cells with "DEPTH" OR "SPAN"
omega_df = omega_df[~omega_df['depth'].str.contains('DEPTH') & ~omega_df['depth'].str.contains('SPAN')]

# save as json for linux reading
# omega_df.to_json('omega_df.json', orient='records')

In [10]:
# import tabula
# tables = tabula.read_pdf("metric tables from Vulcraft OWSJ.pdf", pages='all', multiple_tables=True, pandas_options={'header': None})

In [131]:
# Start working with the saved dataframes for testing
canam_df = pd.read_json("canam_df.json")
omega_df = pd.read_json("omega_df.json")
vulcraft_df = pd.read_json("vulcraft_df.json")
canam_df.name = "canam"
omega_df.name = "omega"
vulcraft_df.name = "vulcraft"

OWSJ_dfs = [canam_df, omega_df, vulcraft_df]

span = 6000
depth = 500
f_load = 8

In [132]:
#Run through each dataframe of tables and pull out the expected weight kg/m of joist used
def find_adjacent_numbers(numbers, target):
    """
    Find numbers adjacent to target number for the sake of running interpolation
    """
    sorted_numbers = np.sort(numbers) # Sort the list of numbers
    index = np.searchsorted(sorted_numbers, target) # Find the index of the target number
    # Get the two numbers adjacent to the target number
    if index == 0:
        return sorted_numbers[0], sorted_numbers[0]
    elif index == len(sorted_numbers):
        return sorted_numbers[-1], sorted_numbers[-1]
    else:
        return sorted_numbers[index - 1], sorted_numbers[index]
        
def isolate_sw(cell):
    sw = float(cell.split("\n")[0])
    return sw


for df in OWSJ_dfs:
    name = df.name
    print(name)
    span_mask = (df['span'] < 1.05*span) & (df['span'] > 0.95*span) 
    depth_mask = (df['depth'] < 1.05*depth) & (df['depth'] > 0.95*depth)
    df = df.loc[span_mask & depth_mask]
    #Omega
    if name == "omega":
        cols = [col for col in df.columns if '/' in col]
        df_sliced = df[cols]
        chords = []
        sws = []
        for column in df_sliced.columns:
            for index, cell_value in enumerate(df_sliced[column]):
                try:
                    f = float(cell_value.split("\n")[0])
                    sw = float(cell_value.split("\n")[2])
                    if (f > 0.97*f_load) & (f < 1.03*f_load):
                        chords.append((column, sw))
                        sws.append(sw)
                except:
                    continue
        if sws == []:
            expected_sw_omega = "no values found in that range"
        else:
            expected_sw_omega = np.mean(sws)
    #Vulcraft         
    elif name == "vulcraft":
        df = df[df['value'].str.contains('Self')] #drop L/360 row
        col_nums = [float(col) for col in df.columns if "." in col] #convert factored headers to float
        adjacent_f_loads = find_adjacent_numbers(col_nums, f_load) #find interpolated, likely average self weight
        df = df[[str(min(adjacent_f_loads)), str(max(adjacent_f_loads))]]
        if adjacent_f_loads[1] == adjacent_f_loads[0]:
            expected_sw_vulcraft = df[str(adjacent_f_loads[0])].values.mean()
        else:
            ratio = (f_load - min(adjacent_f_loads))/(abs(adjacent_f_loads[1] - adjacent_f_loads[0]))
            df[str(f_load)] = (df[str(max(adjacent_f_loads))] - df[str(min(adjacent_f_loads))])*ratio + df[str(min(adjacent_f_loads))]
            expected_sw_vulcraft = df[str(f_load)].values.mean()
    #Canam
    elif name == "canam":
        col_nums = [float(col) for col in df.columns if "." in col] #convert factored headers to float
        adjacent_f_loads = find_adjacent_numbers(col_nums, f_load) #find interpolated, likely average self weight
        df = df[[str(min(adjacent_f_loads)), str(max(adjacent_f_loads))]]
        df = df.map(isolate_sw)
        if adjacent_f_loads[1] == adjacent_f_loads[0]:
            expected_sw_canam = df[str(adjacent_f_loads[0])].values.mean()
        else:
            ratio = (f_load - min(adjacent_f_loads))/(abs(adjacent_f_loads[1] - adjacent_f_loads[0]))
            df[str(f_load)] = (df[str(max(adjacent_f_loads))] - df[str(min(adjacent_f_loads))])*ratio + df[str(min(adjacent_f_loads))]
            expected_sw_canam = df[str(f_load)].values.mean()

canam

omega

vulcraft

In [133]:
print(f'{expected_sw_canam=}, {expected_sw_vulcraft=}, {expected_sw_omega=}')

expected_sw_canam=12.866666666666667, expected_sw_vulcraft=9.2, expected_sw_omega='no values found in that range'

In [134]:
chords

[]

In [59]:
span_mask = (vulcraft_df['span'] < 1.05*span) & (vulcraft_df['span'] > 0.95*span) 
depth_mask = (vulcraft_df['depth'] < 1.05*depth) & (vulcraft_df['depth'] > 0.95*depth) 
df = vulcraft_df.loc[span_mask & depth_mask]
df = df[df['value'].str.contains('Self')] #drop L/360 row
df

,span,depth,value,4.5,6.3,8.1,9.9,11.7,13.5,15.3,17.1,18.9,20.7,22.5,24.3
258,17000,914,Self Wt (kg/m),15.9,18.4,21.0,25.7,26.3,29.0,30.8,34.0,37.9,40.0,45.3,46.0
274,18000,914,Self Wt (kg/m),16.1,20.7,20.9,26.5,28.6,31.0,34.4,38.1,41.7,46.2,51.5,53.2


In [18]:
canam_df.loc[span_mask & depth_mask]

,span,depth,4.5,6.0,7.5,9.0,10.5,12.0,13.5,15.0,16.5,18.0,19.5,21.0,22.5
99,17000,900,15.1\n1.9\n1.00,17.4\n2.3\n1.00,19.1\n2.6\n1.00,21.6\n3.0\n1.00,24.4\n3.3\n1.00,26.5\n3.7\n1.00,29.4\n4.1\n1.00,31.5\n4.4\n1.00,34.3\n4.8\n1.00,36.2\n5.1\n1.13,38.3\n5.4\n1.15,41.7\n5.9\n1.14,46.0\n6.7\n1.18
105,18000,900,16.3\n1.8\n1.00,19.3\n2.2\n1.00,22.0\n2.6\n1.00,24.8\n2.9\n1.00,28.0\n3.3\n1.00,30.5\n3.6\n1.00,33.4\n4.0\n1.00,37.5\n4.5\n1.03,38.8\n4.8\n1.17,44.9\n5.5\n1.21,45.7\n5.7\n1.17,49.5\n6.1\n1.24,52.9\n6.6\n1.26
